# QTAU - Quantum Task Automation Utility
## Visualization Examples

This notebook demonstrates various use cases of QTAU (Quantum-HPC middleware framework) with result visualizations.

QTAU provides a unified interface for managing heterogeneous computational resources, bridging quantum computing and classical HPC systems.

### Table of Contents
1. [Setup and Installation](#setup)
2. [Basic Task Distribution](#basic-tasks)
3. [PennyLane Quantum Circuit Execution](#pennylane)
4. [Throughput Benchmarking](#throughput)
5. [Metrics Analysis](#metrics)
6. [Hybrid Quantum-Classical Optimization](#hybrid)

## 1. Setup and Installation <a id="setup"></a>

In [ ]:
# Import required libraries
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# QTAU imports
from qtau.pilot_compute_service import PilotComputeService, ExecutionEngine

# Quantum computing imports
import pennylane as qml
from pennylane import numpy as pnp

print("All imports successful!")

In [ ]:
# Configuration
RESOURCE_URL = "ssh://localhost"  # Use SSH for local execution
WORKING_DIRECTORY = os.path.join(os.environ["HOME"], "work", "qtau_notebook")

# Create working directory if it doesn't exist
os.makedirs(WORKING_DIRECTORY, exist_ok=True)

print(f"Working directory: {WORKING_DIRECTORY}")

## 2. Basic Task Distribution <a id="basic-tasks"></a>

This example demonstrates distributing simple computational tasks across multiple workers and visualizing the execution times.

In [ ]:
# Define pilot configuration for Dask execution engine
pilot_compute_description_dask = {
    "resource": RESOURCE_URL,
    "working_directory": WORKING_DIRECTORY,
    "type": "dask",
    "number_of_nodes": 1,
    "cores_per_node": 4,
}

# Define pilot configuration for Ray execution engine
pilot_compute_description_ray = {
    "resource": RESOURCE_URL,
    "working_directory": WORKING_DIRECTORY,
    "type": "ray",
    "number_of_nodes": 1,
    "cores_per_node": 4,
}

In [ ]:
# Define computational tasks
def compute_square(x):
    """Compute square of a number with simulated delay."""
    time.sleep(0.1)  # Simulate computation time
    return x ** 2

def compute_fibonacci(n):
    """Compute nth Fibonacci number."""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

def matrix_multiply(size):
    """Multiply two random matrices of given size."""
    A = np.random.rand(size, size)
    B = np.random.rand(size, size)
    return np.dot(A, B).sum()

In [ ]:
# Example: Run tasks and collect timing data
# Note: This is a simulation since we can't run actual distributed tasks in the notebook
# In production, you would use the PilotComputeService as shown in the actual examples

# Simulated execution times for visualization
np.random.seed(42)
n_tasks = 20

# Simulate distributed execution times
task_ids = [f"task-{i}" for i in range(n_tasks)]
submit_times = np.cumsum(np.random.exponential(0.05, n_tasks))
wait_times = np.random.exponential(0.1, n_tasks)
execution_times = np.random.exponential(0.5, n_tasks)
completion_times = submit_times + wait_times + execution_times

# Create a DataFrame for analysis
task_df = pd.DataFrame({
    'task_id': task_ids,
    'submit_time': submit_times,
    'wait_time': wait_times,
    'execution_time': execution_times,
    'completion_time': completion_times,
    'status': ['SUCCESS'] * n_tasks
})

task_df.head(10)

In [ ]:
# Visualize task execution timeline
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Task Timeline (Gantt-style chart)
ax1 = axes[0, 0]
colors = plt.cm.viridis(np.linspace(0, 1, n_tasks))
for i, (_, row) in enumerate(task_df.iterrows()):
    # Wait time (lighter color)
    ax1.barh(i, row['wait_time'], left=row['submit_time'], 
             color=colors[i], alpha=0.3, label='Wait' if i == 0 else '')
    # Execution time (full color)
    ax1.barh(i, row['execution_time'], left=row['submit_time'] + row['wait_time'], 
             color=colors[i], alpha=0.9, label='Execute' if i == 0 else '')

ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Task Index')
ax1.set_title('Task Execution Timeline')
ax1.legend(['Wait Time', 'Execution Time'], loc='lower right')

# 2. Execution Time Distribution
ax2 = axes[0, 1]
ax2.hist(task_df['execution_time'], bins=10, color='steelblue', edgecolor='black', alpha=0.7)
ax2.axvline(task_df['execution_time'].mean(), color='red', linestyle='--', 
            label=f'Mean: {task_df["execution_time"].mean():.3f}s')
ax2.set_xlabel('Execution Time (seconds)')
ax2.set_ylabel('Frequency')
ax2.set_title('Execution Time Distribution')
ax2.legend()

# 3. Wait Time vs Execution Time
ax3 = axes[1, 0]
scatter = ax3.scatter(task_df['wait_time'], task_df['execution_time'], 
                      c=task_df.index, cmap='viridis', s=100, alpha=0.7)
ax3.set_xlabel('Wait Time (seconds)')
ax3.set_ylabel('Execution Time (seconds)')
ax3.set_title('Wait Time vs Execution Time')
plt.colorbar(scatter, ax=ax3, label='Task Order')

# 4. Cumulative Completion
ax4 = axes[1, 1]
sorted_completion = np.sort(task_df['completion_time'])
ax4.step(sorted_completion, np.arange(1, n_tasks + 1), where='post', 
         color='green', linewidth=2)
ax4.fill_between(sorted_completion, np.arange(1, n_tasks + 1), 
                 step='post', alpha=0.3, color='green')
ax4.set_xlabel('Time (seconds)')
ax4.set_ylabel('Completed Tasks')
ax4.set_title('Cumulative Task Completion')

plt.tight_layout()
plt.savefig('task_execution_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSummary Statistics:")
print(f"Total tasks: {n_tasks}")
print(f"Average wait time: {task_df['wait_time'].mean():.3f}s")
print(f"Average execution time: {task_df['execution_time'].mean():.3f}s")
print(f"Total runtime: {task_df['completion_time'].max():.3f}s")
print(f"Throughput: {n_tasks / task_df['completion_time'].max():.2f} tasks/second")

## 3. PennyLane Quantum Circuit Execution <a id="pennylane"></a>

This example demonstrates executing PennyLane quantum circuits and visualizing the results.

In [ ]:
# Define a parameterized quantum circuit
n_wires = 4
n_layers = 2

dev = qml.device('default.qubit', wires=n_wires, shots=None)

@qml.qnode(dev)
def quantum_circuit(params):
    """Strongly entangling layers circuit."""
    qml.StronglyEntanglingLayers(weights=params, wires=range(n_wires))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_wires)]

# Generate random parameters
shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_wires)
np.random.seed(42)
weights = pnp.random.random(size=shape, requires_grad=True)

print(f"Circuit parameters shape: {shape}")
print(f"Total parameters: {np.prod(shape)}")

In [ ]:
# Execute circuit with different parameter sets and collect results
n_samples = 50
results = []

for i in range(n_samples):
    # Generate random parameters for each sample
    params = pnp.random.random(size=shape)
    expectation_values = quantum_circuit(params)
    results.append({
        'sample': i,
        **{f'Z{j}': float(expectation_values[j]) for j in range(n_wires)}
    })

results_df = pd.DataFrame(results)
results_df.head(10)

In [ ]:
# Visualize quantum circuit results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Expectation values over samples
ax1 = axes[0, 0]
for j in range(n_wires):
    ax1.plot(results_df['sample'], results_df[f'Z{j}'], 
             label=f'⟨Z{j}⟩', alpha=0.7, linewidth=1.5)
ax1.set_xlabel('Sample Index')
ax1.set_ylabel('Expectation Value')
ax1.set_title('PauliZ Expectation Values Across Parameter Samples')
ax1.legend(loc='upper right')
ax1.set_ylim(-1.1, 1.1)
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)

# 2. Distribution of expectation values
ax2 = axes[0, 1]
z_columns = [f'Z{j}' for j in range(n_wires)]
colors = plt.cm.tab10(range(n_wires))
for j, col in enumerate(z_columns):
    ax2.hist(results_df[col], bins=15, alpha=0.5, color=colors[j], 
             label=f'⟨{col}⟩', edgecolor='black', linewidth=0.5)
ax2.set_xlabel('Expectation Value')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Expectation Values')
ax2.legend()

# 3. Correlation heatmap between qubits
ax3 = axes[1, 0]
corr_matrix = results_df[z_columns].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            ax=ax3, vmin=-1, vmax=1, square=True,
            xticklabels=[f'⟨Z{j}⟩' for j in range(n_wires)],
            yticklabels=[f'⟨Z{j}⟩' for j in range(n_wires)])
ax3.set_title('Correlation Between Qubit Expectation Values')

# 4. Box plot of expectation values
ax4 = axes[1, 1]
results_df[z_columns].boxplot(ax=ax4)
ax4.set_xlabel('Qubit')
ax4.set_ylabel('Expectation Value')
ax4.set_title('Expectation Value Statistics by Qubit')
ax4.set_xticklabels([f'⟨Z{j}⟩' for j in range(n_wires)])

plt.tight_layout()
plt.savefig('quantum_circuit_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nStatistics for each qubit's expectation value:")
print(results_df[z_columns].describe())

In [ ]:
# Draw the quantum circuit
print("Quantum Circuit Structure:")
print(qml.draw(quantum_circuit)(weights))

## 4. Throughput Benchmarking <a id="throughput"></a>

This example demonstrates throughput benchmarking across different task counts and visualizes the scaling behavior.

In [ ]:
# Simulated benchmark data (similar to pilot_ray_slurm_throughput.py output)
# In production, this data would come from running actual benchmarks

benchmark_data = {
    'cores': [64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256],
    'tasks': [128, 256, 512, 1024, 128, 256, 512, 1024, 128, 256, 512, 1024],
    'runtime_secs': [2.1, 3.8, 7.2, 14.1, 1.2, 2.1, 3.9, 7.5, 0.8, 1.3, 2.2, 4.1],
}

benchmark_df = pd.DataFrame(benchmark_data)
benchmark_df['throughput'] = benchmark_df['tasks'] / benchmark_df['runtime_secs']
benchmark_df['efficiency'] = benchmark_df['throughput'] / benchmark_df['cores']

benchmark_df

In [ ]:
# Visualize throughput benchmarks
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Throughput vs Task Count (grouped by cores)
ax1 = axes[0, 0]
core_configs = benchmark_df['cores'].unique()
width = 0.25
x = np.arange(len(benchmark_df['tasks'].unique()))
task_counts = sorted(benchmark_df['tasks'].unique())

for i, cores in enumerate(sorted(core_configs)):
    data = benchmark_df[benchmark_df['cores'] == cores]
    throughputs = [data[data['tasks'] == t]['throughput'].values[0] for t in task_counts]
    ax1.bar(x + i * width, throughputs, width, label=f'{cores} cores')

ax1.set_xlabel('Number of Tasks')
ax1.set_ylabel('Throughput (tasks/second)')
ax1.set_title('Throughput vs Task Count by Core Configuration')
ax1.set_xticks(x + width)
ax1.set_xticklabels(task_counts)
ax1.legend(title='Configuration')

# 2. Scaling efficiency
ax2 = axes[0, 1]
for tasks in sorted(benchmark_df['tasks'].unique()):
    data = benchmark_df[benchmark_df['tasks'] == tasks].sort_values('cores')
    ax2.plot(data['cores'], data['throughput'], 'o-', 
             label=f'{tasks} tasks', linewidth=2, markersize=8)

# Add ideal scaling line
ideal_x = np.array([64, 128, 256])
ideal_y = ideal_x * (benchmark_df[benchmark_df['cores'] == 64]['throughput'].mean() / 64)
ax2.plot(ideal_x, ideal_y, 'k--', alpha=0.5, label='Ideal scaling')

ax2.set_xlabel('Number of Cores')
ax2.set_ylabel('Throughput (tasks/second)')
ax2.set_title('Throughput Scaling with Core Count')
ax2.legend()
ax2.set_xscale('log', base=2)
ax2.set_yscale('log', base=2)

# 3. Runtime comparison
ax3 = axes[1, 0]
pivot_runtime = benchmark_df.pivot(index='tasks', columns='cores', values='runtime_secs')
pivot_runtime.plot(kind='bar', ax=ax3, width=0.8)
ax3.set_xlabel('Number of Tasks')
ax3.set_ylabel('Runtime (seconds)')
ax3.set_title('Runtime Comparison Across Configurations')
ax3.legend(title='Cores', loc='upper left')
ax3.tick_params(axis='x', rotation=0)

# 4. Efficiency heatmap
ax4 = axes[1, 1]
pivot_efficiency = benchmark_df.pivot(index='tasks', columns='cores', values='efficiency')
sns.heatmap(pivot_efficiency, annot=True, fmt='.2f', cmap='YlGnBu', ax=ax4)
ax4.set_xlabel('Number of Cores')
ax4.set_ylabel('Number of Tasks')
ax4.set_title('Parallel Efficiency (tasks/sec/core)')

plt.tight_layout()
plt.savefig('throughput_benchmark.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nBenchmark Summary:")
print(f"Max throughput: {benchmark_df['throughput'].max():.1f} tasks/second")
print(f"Best configuration: {benchmark_df.loc[benchmark_df['throughput'].idxmax(), 'cores']} cores")
print(f"Avg efficiency: {benchmark_df['efficiency'].mean():.3f} tasks/sec/core")

## 5. Metrics Analysis <a id="metrics"></a>

This example demonstrates how to analyze the metrics.csv file generated by QTAU during task execution.

In [ ]:
# Create sample metrics data (simulating metrics.csv output)
np.random.seed(42)
n_metrics = 100

base_time = pd.Timestamp('2024-01-15 10:00:00')
submit_times = [base_time + pd.Timedelta(seconds=i * 0.1 + np.random.uniform(0, 0.05)) 
                for i in range(n_metrics)]

metrics_data = {
    'task_id': [f'task-{uuid.uuid4().hex[:8]}' for _ in range(n_metrics)] if 'uuid' in dir() else [f'task-{i:08d}' for i in range(n_metrics)],
    'pilot_scheduled': np.random.choice(['pilot-1', 'pilot-2', 'pilot-3'], n_metrics),
    'submit_time': submit_times,
    'wait_time_secs': np.random.exponential(0.2, n_metrics),
    'staging_time_secs': np.random.exponential(0.05, n_metrics),
    'input_staging_data_size_bytes': np.random.randint(1000, 100000, n_metrics),
    'execution_secs': np.random.exponential(0.5, n_metrics),
    'status': np.random.choice(['SUCCESS', 'SUCCESS', 'SUCCESS', 'SUCCESS', 'FAILED'], n_metrics),
    'error_msg': [None if s == 'SUCCESS' else 'Timeout' for s in np.random.choice(['SUCCESS', 'SUCCESS', 'SUCCESS', 'SUCCESS', 'FAILED'], n_metrics)]
}

# Fix error_msg to match status
for i in range(n_metrics):
    if metrics_data['status'][i] == 'SUCCESS':
        metrics_data['error_msg'][i] = None

metrics_df = pd.DataFrame(metrics_data)
metrics_df['completion_time'] = metrics_df['submit_time'] + pd.to_timedelta(
    metrics_df['wait_time_secs'] + metrics_df['execution_secs'], unit='s')
metrics_df['total_time_secs'] = metrics_df['wait_time_secs'] + metrics_df['execution_secs']

metrics_df.head(10)

In [ ]:
# Visualize metrics analysis
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Task status pie chart
ax1 = axes[0, 0]
status_counts = metrics_df['status'].value_counts()
colors = ['#2ecc71', '#e74c3c']
ax1.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%',
        colors=colors[:len(status_counts)], explode=[0.05] * len(status_counts))
ax1.set_title('Task Success Rate')

# 2. Tasks per pilot
ax2 = axes[0, 1]
pilot_counts = metrics_df['pilot_scheduled'].value_counts()
ax2.bar(pilot_counts.index, pilot_counts.values, color=plt.cm.Set2.colors[:len(pilot_counts)])
ax2.set_xlabel('Pilot')
ax2.set_ylabel('Number of Tasks')
ax2.set_title('Task Distribution Across Pilots')
for i, v in enumerate(pilot_counts.values):
    ax2.text(i, v + 1, str(v), ha='center', va='bottom', fontweight='bold')

# 3. Execution time by pilot
ax3 = axes[0, 2]
metrics_df.boxplot(column='execution_secs', by='pilot_scheduled', ax=ax3)
ax3.set_xlabel('Pilot')
ax3.set_ylabel('Execution Time (seconds)')
ax3.set_title('Execution Time Distribution by Pilot')
plt.suptitle('')  # Remove automatic title

# 4. Time breakdown
ax4 = axes[1, 0]
time_components = ['wait_time_secs', 'staging_time_secs', 'execution_secs']
time_means = metrics_df[time_components].mean()
ax4.bar(range(len(time_components)), time_means.values, 
        color=['#3498db', '#9b59b6', '#e74c3c'])
ax4.set_xticks(range(len(time_components)))
ax4.set_xticklabels(['Wait', 'Staging', 'Execution'])
ax4.set_ylabel('Average Time (seconds)')
ax4.set_title('Average Time Breakdown')
for i, v in enumerate(time_means.values):
    ax4.text(i, v + 0.01, f'{v:.3f}s', ha='center', va='bottom')

# 5. Data staging size distribution
ax5 = axes[1, 1]
ax5.hist(metrics_df['input_staging_data_size_bytes'] / 1024, bins=20, 
         color='steelblue', edgecolor='black', alpha=0.7)
ax5.set_xlabel('Input Data Size (KB)')
ax5.set_ylabel('Frequency')
ax5.set_title('Input Data Size Distribution')
ax5.axvline(metrics_df['input_staging_data_size_bytes'].mean() / 1024, 
            color='red', linestyle='--', label=f'Mean: {metrics_df["input_staging_data_size_bytes"].mean()/1024:.1f} KB')
ax5.legend()

# 6. Execution time vs Data size
ax6 = axes[1, 2]
successful_tasks = metrics_df[metrics_df['status'] == 'SUCCESS']
ax6.scatter(successful_tasks['input_staging_data_size_bytes'] / 1024, 
            successful_tasks['execution_secs'], alpha=0.5, c='steelblue')
ax6.set_xlabel('Input Data Size (KB)')
ax6.set_ylabel('Execution Time (seconds)')
ax6.set_title('Execution Time vs Input Data Size')

plt.tight_layout()
plt.savefig('metrics_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nMetrics Summary:")
print(f"Total tasks: {len(metrics_df)}")
print(f"Success rate: {(metrics_df['status'] == 'SUCCESS').mean() * 100:.1f}%")
print(f"Avg execution time: {metrics_df['execution_secs'].mean():.3f}s")
print(f"Avg wait time: {metrics_df['wait_time_secs'].mean():.3f}s")
print(f"Total runtime: {(metrics_df['completion_time'].max() - metrics_df['submit_time'].min()).total_seconds():.2f}s")

## 6. Hybrid Quantum-Classical Optimization <a id="hybrid"></a>

This example demonstrates a hybrid quantum-classical optimization workflow, similar to the PennyLane hybrid computation example.

In [ ]:
# Define quantum and classical devices
dev_qubit = qml.device("default.qubit", wires=1)

@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
    """Qubit rotation circuit."""
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

def target_function(phi1, phi2):
    """Target function to match."""
    return np.cos(phi1) * np.cos(phi2)

def cost_function(params):
    """Cost function: squared difference between quantum and target."""
    phi1, phi2 = 0.5, 0.8  # Fixed target parameters
    qubit_result = qubit_rotation(params[0], params[1])
    target_result = target_function(phi1, phi2)
    return (qubit_result - target_result) ** 2

In [ ]:
# Run optimization
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = pnp.array([0.01, 0.01], requires_grad=True)
n_steps = 100

# Track optimization progress
cost_history = []
param_history = [params.copy()]

for i in range(n_steps):
    params = opt.step(cost_function, params)
    cost_history.append(float(cost_function(params)))
    param_history.append(params.copy())

param_history = np.array(param_history)
print(f"Initial parameters: [{param_history[0][0]:.4f}, {param_history[0][1]:.4f}]")
print(f"Final parameters: [{params[0]:.4f}, {params[1]:.4f}]")
print(f"Final cost: {cost_history[-1]:.6f}")

In [ ]:
# Visualize optimization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Cost vs training step
ax1 = axes[0, 0]
ax1.plot(range(n_steps), cost_history, 'b-', linewidth=2)
ax1.set_xlabel('Training Step')
ax1.set_ylabel('Cost')
ax1.set_title('Optimization Convergence')
ax1.set_yscale('log')
ax1.axhline(0.001, color='red', linestyle='--', alpha=0.5, label='Convergence threshold')
ax1.legend()

# 2. Parameter evolution
ax2 = axes[0, 1]
ax2.plot(range(n_steps + 1), param_history[:, 0], 'r-', label='θ₁', linewidth=2)
ax2.plot(range(n_steps + 1), param_history[:, 1], 'g-', label='θ₂', linewidth=2)
ax2.set_xlabel('Training Step')
ax2.set_ylabel('Parameter Value')
ax2.set_title('Parameter Evolution During Training')
ax2.legend()

# 3. Cost landscape with optimization path
ax3 = axes[1, 0]
theta1_range = np.linspace(-np.pi, np.pi, 100)
theta2_range = np.linspace(-np.pi, np.pi, 100)
T1, T2 = np.meshgrid(theta1_range, theta2_range)
Z = np.zeros_like(T1)

for i in range(len(theta1_range)):
    for j in range(len(theta2_range)):
        Z[j, i] = cost_function(pnp.array([theta1_range[i], theta2_range[j]]))

contour = ax3.contourf(T1, T2, Z, levels=50, cmap='viridis')
plt.colorbar(contour, ax=ax3, label='Cost')
ax3.plot(param_history[:, 0], param_history[:, 1], 'r.-', markersize=3, linewidth=1, alpha=0.7)
ax3.plot(param_history[0, 0], param_history[0, 1], 'go', markersize=10, label='Start')
ax3.plot(param_history[-1, 0], param_history[-1, 1], 'r*', markersize=15, label='End')
ax3.set_xlabel('θ₁')
ax3.set_ylabel('θ₂')
ax3.set_title('Cost Landscape with Optimization Path')
ax3.legend()

# 4. Comparison: Quantum vs Target
ax4 = axes[1, 1]
phi_range = np.linspace(0, 2*np.pi, 50)
quantum_results = []
target_results = []

for phi in phi_range:
    quantum_results.append(float(qubit_rotation(params[0], phi)))
    target_results.append(target_function(0.5, phi))

ax4.plot(phi_range, quantum_results, 'b-', label='Quantum Circuit', linewidth=2)
ax4.plot(phi_range, target_results, 'r--', label='Target Function', linewidth=2)
ax4.set_xlabel('φ₂')
ax4.set_ylabel('Output Value')
ax4.set_title('Quantum Circuit vs Target Function')
ax4.legend()
ax4.set_xlim(0, 2*np.pi)

plt.tight_layout()
plt.savefig('hybrid_optimization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nOptimization Summary:")
print(f"Initial cost: {cost_function(pnp.array([0.01, 0.01])):.6f}")
print(f"Final cost: {cost_history[-1]:.6f}")
print(f"Cost reduction: {(1 - cost_history[-1]/cost_function(pnp.array([0.01, 0.01]))) * 100:.2f}%")

## Summary

This notebook demonstrated various QTAU use cases with visualizations:

1. **Basic Task Distribution**: Visualized task execution timelines, wait/execution time distributions
2. **PennyLane Quantum Circuits**: Showed expectation value distributions and correlations
3. **Throughput Benchmarking**: Analyzed scaling behavior and parallel efficiency
4. **Metrics Analysis**: Demonstrated how to analyze QTAU-generated metrics
5. **Hybrid Optimization**: Visualized quantum-classical optimization convergence

### Running with Actual QTAU

To run these examples with actual distributed execution:

```python
# Initialize PilotComputeService
pcs = PilotComputeService(
    execution_engine=ExecutionEngine.RAY,  # or ExecutionEngine.DASK
    working_directory=WORKING_DIRECTORY
)

# Create a pilot
pilot = pcs.create_pilot(pilot_compute_description)
pilot.wait()

# Submit tasks
tasks = [pcs.submit_task(my_function, arg) for arg in args]

# Wait and get results
pcs.wait_tasks(tasks)
results = pcs.get_results(tasks)

# Read metrics from CSV
metrics_df = pd.read_csv(os.path.join(pcs.pcs_working_directory, 'metrics.csv'))

# Clean up
pcs.cancel()
```

In [ ]:
# Save all figures for reference
print("Generated visualization files:")
print("- task_execution_visualization.png")
print("- quantum_circuit_results.png")
print("- throughput_benchmark.png")
print("- metrics_analysis.png")
print("- hybrid_optimization.png")